# Generating ML XS, ACE Files, Benchmark Inputs and Serpent Scripts

In [127]:
import pandas as pd
import os
import logging

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None  # default='warn'

import sys
# This allows us to import the nucml utilities
sys.path.append("..")


import nucml.datasets as nuc_data
import nucml.ace.data_utilities as ace_utils
import nucml.model.model_utilities as model_utils

In [128]:
figure_dir = "Figures/"

In [129]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [215]:
import importlib
importlib.reload(nuc_data)
importlib.reload(ace_utils)
importlib.reload(model_utils)
print("Finish re-loading scripts.")

Finish re-loading scripts.


In [216]:
import nucml.exfor.data_utilities as exfor_utils
import nucml.general_utilities as gen_utils
importlib.reload(gen_utils)
importlib.reload(exfor_utils)

<module 'nucml.exfor.data_utilities' from '..\\nucml\\exfor\\data_utilities.py'>

# B0 Dataset

In [32]:
# LOADING DATASET
df_b0, _, _, _, _, to_scale_b0, _ = nuc_data.load_exfor(pedro=True, basic=0)

## Decision Tree

In [212]:
dt_ml_ace_dir = "ml/DT_B0/"

In [213]:
dt_results_filepath = "../ML_EXFOR_neutrons/2_DT/dt_resultsB0.csv"
dt_results = pd.read_csv(dt_results_filepath)

# filter down to just three for testing purposes:
dt_results = model_utils.get_best_models_df(dt_results)

dt_results

,id,max_depth,mss,mslmt_strategy,normalizer,train_mae,train_mse,train_evs,train_mae_m,train_r2,val_mae,val_mse,val_evs,val_mae_m,val_r2,test_mae,test_mse,test_evs,test_mae_m,test_r2,model_path,training_time,scaler_path,mt_strategy,msl,tag
986,681,400.0,2.0,NaN,robust,0.025735,0.010784,0.986006,0.000000,0.986006,0.136552,0.082771,0.893108,0.047504,0.893105,0.135758,0.081491,0.894834,0.047654,0.894834,E:\ML_Models_EXFOR\DT_B0\DT400_MSS2_MSL1_robus...,37.492463,E:\ML_Models_EXFOR\DT_B0\DT400_MSS2_MSL1_robus...,one_hot,1.0,Train
252,74,90.0,10.0,NaN,minmax,0.094541,0.038049,0.948882,0.036003,0.948882,0.118665,0.057489,0.922504,0.046883,0.922504,0.119339,0.058362,0.920611,0.046796,0.920610,E:\ML_Models_EXFOR\DT_B0\DT90_MSS10_MSL7_minma...,37.602498,E:\ML_Models_EXFOR\DT_B0\DT90_MSS10_MSL7_minma...,one_hot,7.0,Val
322,94,99.0,10.0,NaN,robust,0.094423,0.038064,0.948795,0.035960,0.948795,0.120346,0.059165,0.920348,0.047123,0.920347,0.119189,0.057920,0.922201,0.046670,0.922201,E:\ML_Models_EXFOR\DT_B0\DT99_MSS10_MSL7_robus...,37.116284,E:\ML_Models_EXFOR\DT_B0\DT99_MSS10_MSL7_robus...,one_hot,7.0,Test


In [214]:
ace_utils.generate_bench_ml_xs(df_b0, dt_results, "U233-MET-FAST-001", to_scale_b0, dt_ml_ace_dir, reset=True)

In [224]:
ace_utils.generate_serpent_bash("ml/DT_B0/")

## K-Nearest-Neighbors

In [49]:
knn_ml_ace_dir = "ml/KNN_B0/"

In [48]:
knn_results_filepath = "../ML_EXFOR_neutrons/1_KNN/knn_results_B0.csv"
knn_results = pd.read_csv(knn_results_filepath)
knn_results = model_utils.get_best_models_df(knn_results)
knn_results

,id,distance_metric,mt_strategy,normalizer,train_mae,train_mse,train_evs,train_mae_m,train_r2,val_mae,val_mse,val_evs,val_mae_m,val_r2,test_mae,test_mse,test_evs,test_mae_m,test_r2,model_path,training_time,scaler_path,run_name,tag
28,16,euclidean,one_hot,standard,0.025762,0.010797,0.985991,0.0,0.985991,0.120330,0.064944,0.913541,0.042944,0.913538,0.119925,0.063432,0.915093,0.043245,0.915092,E:\ML_Models_EXFOR\KNN_B0\k16_distance_euclide...,5255.165533,E:\ML_Models_EXFOR\KNN_B0\k16_distance_euclide...,k16_distance_euclidean_standard_one_hot_B0_v1,Train
33,16,manhattan,one_hot,standard,0.025762,0.010797,0.985991,0.0,0.985991,0.118842,0.062905,0.916375,0.042591,0.916372,0.118412,0.061384,0.917932,0.042845,0.917931,E:\ML_Models_EXFOR\KNN_B0\k16_distance_manhatt...,5162.847080,E:\ML_Models_EXFOR\KNN_B0\k16_distance_manhatt...,k16_distance_manhattan_standard_one_hot_B0_v1,Train
9,9,manhattan,one_hot,standard,0.025882,0.010964,0.985770,0.0,0.985770,0.117340,0.062017,0.917800,0.041271,0.917800,0.117377,0.061497,0.918433,0.041257,0.918433,E:\ML_Models_EXFOR\KNN_B0\k9_distance_manhatta...,5361.724394,E:\ML_Models_EXFOR\KNN_B0\k9_distance_manhatta...,k9_distance_manhattan_standard_one_hot_B0_v1,Val
37,8,manhattan,one_hot,standard,0.025939,0.011031,0.985707,0.0,0.985707,0.117952,0.062309,0.917605,0.041305,0.917605,0.117185,0.061773,0.916968,0.041113,0.916968,E:\ML_Models_EXFOR\KNN_B0\k8_distance_manhatta...,4762.610942,E:\ML_Models_EXFOR\KNN_B0\k8_distance_manhatta...,k8_distance_manhattan_standard_one_hot_B0_v1,Test


In [51]:
ace_utils.generate_bench_ml_xs(df_b0, knn_results, "U233-MET-FAST-001", to_scale_b0, knn_ml_ace_dir, reset=True)

In [ ]:
ace_utils.generate_serpent_bash("ml/KNN_B0/")